In [1]:
from collections import namedtuple
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward','done'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    


In [2]:
import torch
import torchvision
import gym

import random
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn.functional as F
import gym
from gym.envs.registration import register
register(
   id='FrozenLakeNotSlippery-v0',
   entry_point='gym.envs.toy_text:FrozenLakeEnv',
   kwargs={'map_name' : '4x4', 'is_slippery': False},
   max_episode_steps=100,
   reward_threshold=0.78, # optimum = .8196
)

#env = gym.make('FrozenLake8x8-v0')
#env = gym.make('FrozenLake-v0')
env = gym.make('FrozenLakeNotSlippery-v0')
env.render()


use_cuda = torch.cuda.is_available
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

class q_net(nn.Module):
    def __init__(self):
        super(q_net, self).__init__()
        self.linear1 = nn.Linear(1, 20)
        self.linear2 = nn.Linear(20, 4)
        

    def forward(self, x):
#         print("Q_Net: Input " + "-" *5)
#         print(x.shape)
#         print(x)
#         print("Q_Net: Input " + "-" *5)
        x = x.view(-1,1)
        x = F.tanh(self.linear1(x))
        #x = F.softmax(self.linear2(x), dim=0)
        x = F.tanh(self.linear2(x))
        x = x.view(-1,4)
        #print(x.shape)
        #print(x)
        return x
    
    


SFFF
FHFH
FFFH
HFFG


In [3]:
import gym
import numpy as np
import torch.optim as optim
from torch.distributions import Categorical
import random
random.seed(1999)
import math

# custom weights initialization 
def weights_init(m):
    classname = m.__class__.__name__
    #print classname
    #print q_net
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        if not m.bias is None:
            m.bias.data.normal_(0.0, 0.02)
        #m.weight.data.uniform_(0.0, 0.02)        
        #m.weight.data.fill_(0.1)
        #if not m.bias is None:
        #    m.bias.data.fill_(0.1)
        print m
        


# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mse = nn.MSELoss(reduce=False)
NUM_EPISODES = 1000000
BATCH_SIZE = 500
GAMMA = 0.9
TARGET_UPDATE = 50
EPS_START = 0.9
EPS_END = 0.0
EPS_DECAY = 1000000
online_net = q_net().to(device)
online_net.apply(weights_init)
target_net = q_net().to(device)
target_net.load_state_dict(online_net.state_dict())
target_net.eval()


memory = ReplayMemory(100000)
#optimizer = optim.RMSprop(online_net.parameters(), lr=0.001)
optimizer = optim.Adam(online_net.parameters(), lr=0.0001)

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see http://stackoverflow.com/a/19343/3343043 for
    # detailed explanation).
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    #non_final_mask = torch.tensor(tuple(map(lambda d: d is False,
    #                                      batch.done)), device=device, dtype=torch.uint8).unsqueeze(1)
    # Compute states that are final.
    next_state_final_mask = torch.tensor(tuple(map(lambda d: d in [5,7,11,12],
                                          batch.next_state)), device=device, dtype=torch.uint8).unsqueeze(1) 
    next_state_finak_list = [d for d in batch.next_state if d in [5,7,11,12] ]
    
    
    #non_final_next_states = torch.cat([FloatTensor([s]) for s,d in zip(batch.next_state,batch.done)
    #                                            if d is False])
    
    #state_batch = torch.cat([torch.FloatTensor([s]) for s in batch.state])
    state_batch = FloatTensor(batch.state)
    state_batch = state_batch.view(BATCH_SIZE, 1)
    next_state_batch = FloatTensor(batch.next_state)
    next_state_batch = next_state_batch.view(BATCH_SIZE, 1)
    #action_batch = torch.cat([torch.LongTensor([[a.item()]]) for a in batch.action])
    action_batch = LongTensor(batch.action).view(BATCH_SIZE,1)
    #reward_batch = torch.cat([torch.tensor([r]) for r in batch.reward])
    reward_batch = Tensor(batch.reward).view(BATCH_SIZE,1)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
#     print("state_batch "+"-" * 10)
#     print(state_batch.shape)
#     print("action_batch "+"-" * 10)
#     print(action_batch.shape)
    state_action_values = online_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # next_state_values = torch.zeros(BATCH_SIZE, device=device).view(BATCH_SIZE,1)
#     print("non_final_mask")
#     print(non_final_mask.shape)
    #next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    # DDQN ------------------------
    # Y_ddqn = R_next + GAMMA * Q_target(S_next, argmax_a Q_online(S_next, a) )
    # Below is the argmax_a Q_online(S_next,a)
    argmax_a_online_net_next_state = online_net(next_state_batch).max(1)[1].detach().view(BATCH_SIZE,1)
    
    # Below is the actual Q_target
    next_state_values = target_net(next_state_batch).detach().gather(1, argmax_a_online_net_next_state)
    # DDQN ------------------------
#     print "next_state_values"
#     print next_state_values.shape
#     print next_state_values.type()
#     print "final_mask"
#     print final_mask.shape
#     print  final_mask.type()
#     print "next_state_values[final_mask]"
#     print next_state_values[final_mask].shape
#     print next_state_values[final_mask].type()
    next_state_values[next_state_final_mask] = torch.zeros(len(next_state_finak_list), device=device).view(len(next_state_finak_list))
#     print("next_state_values")
#     print(next_state_values.shape)
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
#     print("expected_state_action_values")
#     print(expected_state_action_values.shape)

    # Compute Huber loss (this is like MSE , but less sensitive to outliers )
    # loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    #loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
    loss = mse( state_action_values, expected_state_action_values)
    debug = False
    if debug:
        print("-" * 40)
        print("States")
        print(state_batch)
        print("Target Q")
        print(expected_state_action_values)
        print(expected_state_action_values.shape)
        print("Actual Q")
        print(state_action_values)
        print(state_action_values.shape)
        print("Loss")
        print(loss)
        print(loss.shape)
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward(torch.ones(len(state_action_values), device=device).unsqueeze(1))
    
    # param in online_net.parameters():
    #    param.grad.data.clamp_(-1, 1)
    optimizer.step()
    
    if debug: 
        print("AFTER OPTIMIZATION:")
        print("New actual Q")
        print(online_net(state_batch).gather(1, action_batch))
    
    

Linear(in_features=1, out_features=20, bias=True)
Linear(in_features=20, out_features=4, bias=True)


In [4]:
score = []
times_trained = 0
times_reach_goal = 0
steps_done = 0


# TO BE DELETED_______
#steps_done = 40000000
#EPS_END = 0.0

# TO BE DELETED_______


for k in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    #observation, reward, done, info = env.step(env.action_space.sample()) # take a random action
    episode_series = []
    reward = 0
    episode_step = 0
    while not done:
        # Get action from pi
        # action = env.action_space.sample()
        np_observation = np.array(observation)
        #np_observation = np.expand_dims(np_observation, axis=0)
        np_observation = np.expand_dims(np_observation, axis=0)
        observation_tensor = FloatTensor(np_observation)
        #print(observation_tensor)
        #net.eval()
        #print("before eval")
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
        steps_done += 1
        q_values = online_net(observation_tensor)
        if sample >= eps_threshold: 
            #print "observation_tensor"
            #print observation_tensor.type()
            
            action = q_values.max(1)[1] # First 1 is the dimension, second 1 is the index (this is argmax)
        else:
            action = torch.LongTensor([[random.randint(0,3)]], device=device)
        
            
        #break
        # Execute action in environment.
        old_state = observation                    
            
        
        observation, reward, done, info = env.step(action.item()) 
        new_state = observation
        
        

        
        
#         if done and reward != 1.0:
#             if episode_step > 50:
#                 reward = -0.2
#             if episode_step > 80:
#                 reward = -0.5
#             if new_state in [5,7,11,12]:
#                 reward = -1.0

        
        
        # Store the transition in memory
        #if old_state != new_state:

        memory.push(old_state, action, new_state, reward, done)
        if k%5000 == 0:
            #print("old_state != new_state")
            #print(old_state != new_state)
            #print("oldstate " + str(old_state) + " newstate " + str(new_state))
            print("q_values ")
            print(q_values)
            print("On state="+ str(old_state) + ", selected action=" + str(action.item()) + " , " + \
              "Random? " + str( sample < eps_threshold ))
            print("new state="+ str(new_state) + ", done="+str(done) + \
             ". Reward: " + str(reward))

        # Perform one step of the optimization (on the target network)
        if k > BATCH_SIZE :
            optimize_model()
            times_trained = times_trained + 1

        episode_step += 1
        #env.render()
    if k % TARGET_UPDATE == 0:
        target_net.load_state_dict(online_net.state_dict())
    
    if k % 1000 ==0:
        print_q_table()
   
    if len(score) < 100:
        score.append(reward)
    else:
        score[k % 100] = reward

    if k%1000 == 0:
        print("Episode {} finished after {} timesteps with r={}. Running score: {}. Times trained: \
              {}. Times reached goal: {}. \
              Steps done: {}. EPS_DECAY: {}. EPS_THRESHOLD: {}.".format(k, len(episode_series), \
                                                                    reward, np.mean(score), times_trained, \
                                                                       times_reach_goal, steps_done, \
                                                                       EPS_DECAY, eps_threshold))
        times_trained = 0
        times_reach_goal = 0
        #print("Game finished. " + "-" * 5)
        #print(len(episode_series))
#         for param in net.parameters():
#             print(param.data)
        

    
    
    
    if reward > 0.0:
        times_reach_goal = times_reach_goal + 1
        
    
    

q_values 
tensor(1.00000e-02 *
       [[ 1.1451,  1.4440,  4.2905,  3.2095]], device='cuda:0')
On state=0, selected action=0 , Random? True
new state=0, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 1.1451,  1.4440,  4.2905,  3.2095]], device='cuda:0')
On state=0, selected action=3 , Random? True
new state=0, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 1.1451,  1.4440,  4.2905,  3.2095]], device='cuda:0')
On state=0, selected action=3 , Random? True
new state=0, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 1.1451,  1.4440,  4.2905,  3.2095]], device='cuda:0')
On state=0, selected action=3 , Random? True
new state=0, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 1.1451,  1.4440,  4.2905,  3.2095]], device='cuda:0')
On state=0, selected action=2 , Random? True
new state=1, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[ 1.0304,  1.3544,  4.7594,  3.2289]], device='cuda:0')
On state=1, selected

NameError: name 'print_q_table' is not defined

In [5]:
def print_q_table():
    for i in range(16):
        st = np.array(i)
        st = np.expand_dims(st, axis=0)
        q_vals = online_net(FloatTensor(st))
        outp = " state (" +str(i) + ") "
        n = 0
        for tensr in q_vals:
            for cell in tensr:
                outp = outp + " A[" + str(n) + "]:(" + str(cell.item()) + ")"
                n += 1
        print(outp)
        
print_q_table()
print_q_table()

 state (0)  A[0]:(-0.016590623185) A[1]:(-0.0379296652973) A[2]:(0.00880737882107) A[3]:(0.0203321184963)
 state (1)  A[0]:(-0.0167372319847) A[1]:(-0.0366442278028) A[2]:(0.0104934358969) A[3]:(0.0212363991886)
 state (2)  A[0]:(-0.0168837141246) A[1]:(-0.0353712290525) A[2]:(0.0121734226122) A[3]:(0.022135829553)
 state (3)  A[0]:(-0.0170289520174) A[1]:(-0.0341202318668) A[2]:(0.0138428043574) A[3]:(0.0230262875557)
 state (4)  A[0]:(-0.0171718932688) A[1]:(-0.0329002924263) A[2]:(0.0154972476885) A[3]:(0.0239038150758)
 state (5)  A[0]:(-0.0173115506768) A[1]:(-0.0317197963595) A[2]:(0.0171326845884) A[3]:(0.0247646775097)
 state (6)  A[0]:(-0.0174470487982) A[1]:(-0.0305862799287) A[2]:(0.018745386973) A[3]:(0.0256054457277)
 state (7)  A[0]:(-0.0175776146352) A[1]:(-0.0295063517988) A[2]:(0.0203320086002) A[3]:(0.0264229979366)
 state (8)  A[0]:(-0.0177025925368) A[1]:(-0.0284855756909) A[2]:(0.0218896120787) A[3]:(0.0272145923227)
 state (9)  A[0]:(-0.0178214609623) A[1]:(-0.027